In [70]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np


In [71]:
#LOAD THE TRAINED MODEL 
model = load_model('model.keras')
#LOAD THE SCALER
with open('scaler.pkl', 'rb') as file:
    scaler= pickle.load(file)
#LOAD THE ENCODER
with open('onehot_encoder.pkl', 'rb') as file:
    label_encoder_geo = pickle.load(file)
#LOAD THE COLUMNS
with open('label_encoder.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)




In [93]:
input_data = { 
    'CreditScore': 600,
    'Age': 22, 
    'Geography': 'France',
    'Gender': 'Male',   
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000,}
    
#example inputs

In [94]:
input_df = pd.DataFrame([input_data])
#CONVERT TO DATAFRAME
geo_encoded = label_encoder_geo.transform(input_df[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=label_encoder_geo.get_feature_names_out(['Geography']))
#ENCODE CATEGORICAL FEATURE
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [95]:
input_df = pd.concat([input_df.drop(columns=['Geography']), geo_encoded_df], axis=1)
#DROP THE ORIGINAL CATEGORICAL COLUMN AND CONCATENATE THE ENCODED COLUMNS
input_df


,CreditScore,Age,Gender,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,22,Male,3,60000,2,1,1,50000,1.0,0.0,0.0


In [96]:
input_df

,CreditScore,Age,Gender,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,22,Male,3,60000,2,1,1,50000,1.0,0.0,0.0


In [97]:
if input_df['Gender'].dtype == object:
    input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])

In [100]:
# Ensure input_df columns match the scaler's expected columns and order
expected_columns = scaler.feature_names_in_
input_df = input_df[expected_columns]

In [101]:
input_scaled = scaler.transform(input_df)

In [102]:
input_scaled

array([[-0.53598516,  0.91324755, -1.60651026, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [103]:
#predict
prediction = model.predict(input_scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step


In [104]:
prediction


array([[0.00572633]], dtype=float32)

In [106]:
if prediction[0][0] > 0.5:
    print("The customer is likely to churn.")
else:
    print("The customer is not likely to churn.")

The customer is not likely to churn.
